In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=True)

In [3]:
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 4  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 4  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout).to(device)

In [4]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len, torch.tensor(site_to_ids[MASK_SITE]).to(device))
trainer.train(epochs=4000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


Traceback (most recent call last):
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 192, in train
    self.train_step()
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 154, in train_step
    loss_volume = self.get_lattice_loss("train")
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 144, in get_lattice_loss
    predicted_volume = self.model(self.torch_datasets[dataset_name]["spacegroup_number"],
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/kna/WyckoffTransformer/wyckoff_transformer.py", line 74, in forward
    output = self.transformer_encoder(src,

known_seq_len,▃█▄▁
lr,▁▁▁▁
train_loss_batch,▁▂█▄
train_loss_element,█▁▃▆
train_loss_site,▇▁▇█
train_loss_volume,▁▂█▄
known_seq_len,1
lr,5.0
train_loss_batch,121.70027
train_loss_element,4.67258
train_loss_site,4.96429


OutOfMemoryError: CUDA out of memory. Tried to allocate 456.00 MiB. GPU 0 has a total capacty of 44.52 GiB of which 372.69 MiB is free. Including non-PyTorch memory, this process has 44.15 GiB memory in use. Of the allocated memory 37.83 GiB is allocated by PyTorch, and 5.82 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF